### `(a) So sánh hai thuật toán.`

#### <div align="center"><strong>Giải thích bằng toán học</strong></div>

#### Xét hai thuật toán

1. **Thuật toán 1 (Duyệt tuần tự từ đầu mảng)**  
   - Khởi tạo `i = 1`.  
   - Kiểm tra từng phần tử từ **chỉ số 1 đến n**.  
   - Nếu gặp phần tử **bằng 1**, trả về vị trí đó.  

2. **Thuật toán 2 (Chọn ngẫu nhiên một vị trí trong mảng)**  
   - Chọn ngẫu nhiên một vị trí `i` từ tập `[1, n]`.  
   - Nếu phần tử tại vị trí đó bằng 1, trả về vị trí đó.  
   - Nếu không, tiếp tục chọn ngẫu nhiên đến khi gặp số 1.  

#### Phân tích độ phức tạp thuật toán

1. **Thuật toán 1**
   - Vì có nửa số phần tử là 1, trung bình thuật toán sẽ cần duyệt qua **n/2 phần tử** để tìm thấy số 1 đầu tiên.
   - Trường hợp tốt nhất: Số 1 nằm ở vị trí đầu tiên (1 lần kiểm tra) => `0(1)`.
   - Trường hợp xấu nhất: Số 1 nằm ở cuối cùng (n lần kiểm tra) => `O(n)`.
   - Trung bình: Toàn bộ số 0 nằm ở đầu mảng => `O(n/2) = O(n)`

2. **Thuật toán 2**
   - Xác suất chọn một vị trí chứa số 1
   - Xác suất chọn một vị trí chứa số 1 là $ \frac{1}{2} $, vì nửa mảng là số 1.
   - Do đó, số lần lặp trung bình để chọn được số 1 là:

   $$
   E(X) = \sum_{k=1}^{\infty} k \cdot P(X = k)
   $$

      - Xác suất tìm được số 1
      - Xác suất tìm được số 1 ngay lần đầu: $ \frac{1}{2} $.
      - Xác suất tìm được số 1 ở lần thứ hai: $ \left(\frac{1}{2}\right) \cdot \left(\frac{1}{2}\right) $.
      - Xác suất tìm được số 1 ở lần thứ $ k $: $ \left(\frac{1}{2}\right)^k $.
      - Tổng kỳ vọng số lần thử cần thiết

   $$
   E(X) = \sum_{k=1}^{\infty} k \cdot \left(\frac{1}{2}\right)^k
   $$

   - Công thức tổng quát của chuỗi này là:

   $$
   E(X) = 2
   $$

   - Tức là trung bình chỉ mất **2 lần chọn** để tìm ra số 1.
   - Như vậy, **thuật toán 2** có thời gian trung bình `O(1)`, nhanh hơn nhiều so với **thuật toán 1**  `O(n)`.

#### Ổn định và tính chắc chắn  
1. **Thuật toán 1** luôn tìm được số 1 sau tối đa `O(n)` bước.  
2. **Thuật toán 2** nhanh hơn trung bình, nhưng **không đảm bảo** hoàn thành trong thời gian nhất định.  



### `(b) Thuật toán nào tốt hơn?`

| Tiêu chí | Thuật toán 1 | Thuật toán 2 |
|----------|-------------|-------------|
| **Thời gian trung bình** | `O(n)` | `O(1)` |
| **Thời gian tệ nhất** | `O(n)` | `Vô hạn` |
| **Ổn định** | Luôn tìm thấy trong `O(n)` | Rủi ro chọn sai vô hạn |
| **Dễ triển khai** | Duyệt tuần tự | Dùng random |

#### <div align="center"><strong>Phương pháp mô phỏng</strong></div>

In [14]:
def algorithm_1(a):
    i = 0
    while a[i] != 1:
        i += 1
    return i

def algorithm_2(a):
    n = len(a)
    while True:
        i = random.randint(0, n - 1)
        if a[i] == 1:
            return i

def compare_algorithms(n, num_simulations=1000):
    a = [0] * (n // 2) + [1] * (n // 2)
    random.shuffle(a)

    time_1 = sum(algorithm_1(a) for _ in range(num_simulations)) / num_simulations
    time_2 = sum(algorithm_2(a) for _ in range(num_simulations)) / num_simulations

    return time_1, time_2

n = 100
time_1, time_2 = compare_algorithms(n)
print(f"Thời gian trung bình thuật toán 1: {time_1}")
print(f"Thời gian trung bình thuật toán 2: {time_2}")

Thời gian trung bình thuật toán 1: 3.0
Thời gian trung bình thuật toán 2: 52.816


#### <div align="center"><strong>Kết luận</strong></div>

Qua các phân tích và mô phỏng trên:

- **Thuật toán 2** nhanh hơn trong hầu hết các trường hợp.  
- **Thuật toán 1** an toàn hơn và luôn chạy được trong thời gian giới hạn.  
- **Gợi ý tối ưu**: Kết hợp cả hai, bắt đầu bằng random, sau `k` lần thử chưa thành công thì chuyển sang duyệt tuần tự. 